##### Copyright 2025 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# tsp

<table align="left">
<td>
<a href="https://colab.research.google.com/github/google/or-tools/blob/main/examples/notebook/constraint_solver/tsp.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/colab_32px.png"/>Run in Google Colab</a>
</td>
<td>
<a href="https://github.com/google/or-tools/blob/main/ortools/constraint_solver/samples/tsp.py"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/github_32px.png"/>View source on GitHub</a>
</td>
</table>

First, you must install [ortools](https://pypi.org/project/ortools/) package in this colab.

In [1]:
%pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0



Simple Travelling Salesman Problem.

A description of the problem can be found here:
http://en.wikipedia.org/wiki/Travelling_salesperson_problem.



In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp



def create_data_model():
    """Stores the data for the problem."""
    data = {}
    # Locations in block units
    locations = [
        # fmt:off
      (4, 4),  # depot
      (2, 0), (8, 0),  # locations to visit
      (0, 1), (1, 1),
      (5, 2), (7, 2),
      (3, 3), (6, 3),
      (5, 5), (8, 5),
      (1, 6), (2, 6),
      (3, 7), (6, 7),
      (0, 8), (7, 8)
        # fmt:on
    ]
    # Convert locations in meters using a city block dimension of 114m x 80m.
    data["locations"] = [(l[0] * 114, l[1] * 80) for l in locations]
    data["num_vehicles"] = 1
    data["depot"] = 0
    return data


def create_distance_callback(data, manager):
    """Creates callback to return distance between points."""
    distances_ = {}
    index_manager_ = manager
    # precompute distance between location to have distance callback in O(1)
    for from_counter, from_node in enumerate(data["locations"]):
        distances_[from_counter] = {}
        for to_counter, to_node in enumerate(data["locations"]):
            if from_counter == to_counter:
                distances_[from_counter][to_counter] = 0
            else:
                distances_[from_counter][to_counter] = abs(
                    from_node[0] - to_node[0]
                ) + abs(from_node[1] - to_node[1])

    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = index_manager_.IndexToNode(from_index)
        to_node = index_manager_.IndexToNode(to_index)
        return distances_[from_node][to_node]

    return distance_callback


def print_solution(manager, routing, assignment):
    """Prints assignment on console."""
    print(f"Objective: {assignment.ObjectiveValue()}")
    index = routing.Start(0)
    plan_output = "Route for vehicle 0:\n"
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += f" {manager.IndexToNode(index)} ->"
        previous_index = index
        index = assignment.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += f" {manager.IndexToNode(index)}\n"
    plan_output += f"Distance of the route: {route_distance}m\n"
    print(plan_output)


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["locations"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    distance_callback = create_distance_callback(data, manager)
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    assignment = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if assignment:
        print_solution(manager, routing, assignment)


main()



Time window priority

In [3]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    """Define problem data with priority node."""
    data = {}
    # Include Node 4 as a clone of Node 3 for multiple visits
    data['distance_matrix'] = [
        [0, 10, 15, 20, 20],
        [10, 0, 35, 25, 25],
        [15, 35, 0, 30, 30],
        [20, 25, 30, 0, 0],
        [20, 25, 30, 0, 0]
    ]
    data['time_matrix'] = [
        [0, 5, 10, 15, 15],
        [5, 0, 20, 10, 10],
        [10, 20, 0, 25, 25],
        [15, 10, 25, 0, 0],
        [15, 10, 25, 0, 0]
    ]
    data['demands'] = [0, 100, 150, 100, 100]  # Node 3 split into 100 + 100
    data['vehicle_capacities'] = [400, 400]
    data['num_vehicles'] = 2
    data['depot'] = 0
    # Add time windows: [start, end] in minutes
    data['time_windows'] = [
        (0, 1000),  # Depot: wide window
        (0, 30),    # Node 1: priority, visit within 30 min
        (0, 1000),  # Node 2: no priority
        (0, 1000),  # Node 3: no priority
        (0, 1000)   # Node 4 (Node 3 clone): no priority
    ]
    return data

def print_solution(data, manager, routing, solution):
    """Print routes, loads, and times."""
    total_distance = 0
    total_time = 0
    total_load = 0
    time_dimension = routing.GetDimensionOrDie('Time')
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = f'Route for vehicle {vehicle_id}:\n'
        route_distance = 0
        route_time = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            time_var = time_dimension.CumulVar(index)
            plan_output += f' {node_index} Load({route_load}) Time({solution.Min(time_var)}-{solution.Max(time_var)}) -> '
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
            route_time += data['time_matrix'][manager.IndexToNode(previous_index)][manager.IndexToNode(index)]
        node_index = manager.IndexToNode(index)
        time_var = time_dimension.CumulVar(index)
        plan_output += f' {node_index} Load({route_load}) Time({solution.Min(time_var)}-{solution.Max(time_var)})\n'
        plan_output += f'Distance: {route_distance} units, Time: {route_time} min, Load: {route_load}/{data["vehicle_capacities"][vehicle_id]}\n'
        print(plan_output)
        total_distance += route_distance
        total_time += route_time
        total_load += route_load
    print(f'Total Distance: {total_distance} units')
    print(f'Total Time: {total_time} min')
    print(f'Total Load Utilization: {total_load / sum(data["vehicle_capacities"]):.2%}')

def main():
    """Solve the VRP with priority nodes."""
    data = create_data_model()
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)

    # Distance callback
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    transit_distance_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_distance_index)

    # Time callback
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]
    transit_time_index = routing.RegisterTransitCallback(time_callback)
    time_dimension_name = 'Time'
    routing.AddDimension(
        transit_time_index,
        30,   # Slack: allow waiting time
        1000, # Maximum time per vehicle
        True, # Start cumul to zero
        time_dimension_name
    )
    time_dimension = routing.GetDimensionOrDie(time_dimension_name)

    # Add time window constraints
    for node in range(len(data['time_windows'])):
        index = manager.NodeToIndex(node)
        time_dimension.CumulVar(index).SetRange(data['time_windows'][node][0], data['time_windows'][node][1])

    # Demand callback
    def demand_callback(from_index):
        return data['demands'][manager.IndexToNode(from_index)]
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index, 0, data['vehicle_capacities'], True, 'Capacity')

    # Encourage high load utilization
    for vehicle_id in range(data['num_vehicles']):
        routing.AddVariableMaximizedByFinalizer(
            routing.GetDimensionOrDie('Capacity').CumulVar(routing.End(vehicle_id)))

    # Solve
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    search_parameters.time_limit.seconds = 30
    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        print_solution(data, manager, routing, solution)

if __name__ == '__main__':
    main()

Route for vehicle 0:
 0 Load(0) Time(0-0) ->  2 Load(150) Time(10-40) ->  0 Load(150) Time(20-80)
Distance: 30 units, Time: 20 min, Load: 150/400

Route for vehicle 1:
 0 Load(0) Time(0-0) ->  1 Load(100) Time(5-30) ->  4 Load(200) Time(15-70) ->  3 Load(300) Time(15-100) ->  0 Load(300) Time(30-145)
Distance: 55 units, Time: 30 min, Load: 300/400

Total Distance: 85 units
Total Time: 50 min
Total Load Utilization: 56.25%


Vehicle allocated to priority node(s)

In [4]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    """Define problem data without node cloning."""
    data = {}
    # Original 4 nodes: Depot (0), Customers (1, 2, 3)
    data['distance_matrix'] = [
        [0, 10, 15, 20],
        [10, 0, 35, 25],
        [15, 35, 0, 30],
        [20, 25, 30, 0]
    ]
    data['time_matrix'] = [
        [0, 5, 10, 15],
        [5, 0, 20, 10],
        [10, 20, 0, 25],
        [15, 10, 25, 0]
    ]
    data['demands'] = [0, 100, 150, 200]  # Node 3 demand is 200 (no cloning)
    data['vehicle_capacities'] = [400, 400]
    data['num_vehicles'] = 2
    data['depot'] = 0
    return data

def print_solution(data, manager, routing, solution):
    """Print routes, loads, and times."""
    total_distance = 0
    total_time = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = f'Route for vehicle {vehicle_id}:\n'
        route_distance = 0
        route_time = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += f' {node_index} Load({route_load}) -> '
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
            route_time += data['time_matrix'][manager.IndexToNode(previous_index)][manager.IndexToNode(index)]
        node_index = manager.IndexToNode(index)
        plan_output += f' {node_index} Load({route_load})\n'
        plan_output += f'Distance: {route_distance} units, Time: {route_time} min, Load: {route_load}/{data["vehicle_capacities"][vehicle_id]}\n'
        print(plan_output)
        total_distance += route_distance
        total_time += route_time
        total_load += route_load
    print(f'Total Distance: {total_distance} units')
    print(f'Total Time: {total_time} min')
    print(f'Total Load Utilization: {total_load / sum(data["vehicle_capacities"]):.2%}')

def main():
    """Solve the VRP with vehicle priority for Node 1."""
    data = create_data_model()
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)

    # Distance callback
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    transit_distance_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_distance_index)

    # Time callback
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]
    transit_time_index = routing.RegisterTransitCallback(time_callback)
    routing.AddDimension(transit_time_index, 0, 1000, True, 'Time')

    # Demand callback
    def demand_callback(from_index):
        return data['demands'][manager.IndexToNode(from_index)]
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index, 0, data['vehicle_capacities'], True, 'Capacity')

    # Assign Node 1 to Vehicle 0 only
    routing.SetAllowedVehiclesForIndex([0], manager.NodeToIndex(1))

    # Encourage high load utilization
    for vehicle_id in range(data['num_vehicles']):
        routing.AddVariableMaximizedByFinalizer(
            routing.GetDimensionOrDie('Capacity').CumulVar(routing.End(vehicle_id)))

    # Solve
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    search_parameters.time_limit.seconds = 30
    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print("No solution found.")

if __name__ == '__main__':
    main()

Route for vehicle 0:
 0 Load(0) ->  1 Load(100) ->  0 Load(100)
Distance: 20 units, Time: 10 min, Load: 100/400

Route for vehicle 1:
 0 Load(0) ->  2 Load(150) ->  3 Load(350) ->  0 Load(350)
Distance: 65 units, Time: 50 min, Load: 350/400

Total Distance: 85 units
Total Time: 60 min
Total Load Utilization: 56.25%
